In [22]:
# import packages
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pdz
import numpy as np
import pandas as pd

from pandas import Series, DataFrame
import Bio
from Bio import SeqIO,AlignIO

import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.mixture import GaussianMixture as GMM

from sklearn.manifold import TSNE
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import copy
import warnings
warnings.filterwarnings('ignore')

# methods

### 
### This includes code copied and pasted from the main methods used for the website in BioKlustering-Website/BioKlustering/mlmodel/parser/GMM.py
### These methods are copy-pasted instead of directly included due to difficulties importing Django classes for running locally without running the server
###

def parseFasta(data):
    d = {fasta.id : str(fasta.seq) for fasta in SeqIO.parse(data, "fasta")}
    pd.DataFrame([d])
    s = pd.Series(d, name='Sequence')
    s.index.name = 'ID'
    s.reset_index()
    return pd.DataFrame(s)

def get_kmer_table(path,k_min,k_max):
    genes, gene_len, output_df = read_fasta(path)
    count_vect = CountVectorizer(analyzer='char', ngram_range=(k_min, k_max))
    X = count_vect.fit_transform(genes)
    chars = count_vect.get_feature_names()
    kmers = X.toarray()
    kmer_freq = []
    for i in range(len(genes)):
        kmer_freq.append(kmers[i] / gene_len[i])
    input = pd.DataFrame(kmer_freq, columns=chars)
    return input, output_df

def get_gene_sequences(filename):
    genes = []
    for record in SeqIO.parse(filename, "fasta"):
        genes.append(str(record.seq))
    return genes

# genes: a list of gene sequences, which can directly be generated from get_gene_sequences().
def get_gene_len(genes):
    gene_len = []

    for i in range(len(genes)):
        gene_len.append(len(genes[i]))
    return gene_len

#read single fasta file containing all the gene sequences
def read_fasta(path):
    all_genes = []
    all_gene_len = []
    output_df = pd.DataFrame()

    virus = parseFasta(path)
    output_df = pd.concat([output_df, virus])
    virus = virus.drop_duplicates(keep="last")
    genes = list(virus['Sequence'])
    genes_seq = get_gene_sequences(path)
    gene_len = get_gene_len(genes_seq)
    all_genes = all_genes + genes_seq
    all_gene_len = all_gene_len + gene_len
    return all_genes, all_gene_len, output_df

def get_predictions_default(path,k_min,k_max,num_class,cov_type):
    seed  = np.random.seed(None)
    ran_state = np.random.get_state()
    kmer_table, output_df = get_kmer_table(path, k_min, k_max)
    gmm = GMM(n_components=num_class,covariance_type=cov_type,random_state = seed).fit(kmer_table)
    labels = gmm.predict(kmer_table)
    return labels,ran_state

def get_predictions_from_state(path,k_min,k_max,num_class,cov_type,state):
    kmer_table, output_df = get_kmer_table(path, k_min, k_max)
    gmm = GMM(n_components=num_class,covariance_type=cov_type,random_state = np.random.set_state(state)).fit(kmer_table)
    labels = gmm.predict(kmer_table)
    return labels

def get_predictions(path,k_min,k_max,num_class,cov_type, seed):
    kmer_table, output_df = get_kmer_table(path, k_min, k_max)
    gmm = GMM(n_components=num_class, covariance_type=cov_type, random_state=seed).fit(kmer_table)
    predictions = gmm.predict(kmer_table)
    output_df.insert(0, "Labels", predictions)
    return predictions

def cal_accuracy(labels, predictions):
    err = 0
    total_len = len(labels)
    for i in range(len(labels)):
        if (labels[i] == -1):
            total_len = total_len-1
            continue
        if (labels[i] != predictions[i]):
            err += 1
            
    return 1-err/(total_len)

def get_predictions_semi(path,k_min,k_max,num_class,cov_type,seed,labels):
    targets = []
    unique_given_labels = get_unique_numbers(labels)
    if num_class < len(unique_given_labels) - 1 and -1 in unique_given_labels:
        num_class = len(unique_given_labels) - 1
    if num_class < len(unique_given_labels) and -1 not in unique_given_labels:
        num_class = len(unique_given_labels)
    kmer_table, output_df = get_kmer_table(path, k_min, k_max)

    finalDf = pd.concat([kmer_table, labels], axis=1)
    gmm = GMM(n_components=num_class, covariance_type=cov_type, random_state=seed)
    for i in range(num_class):
        if i in list(finalDf.Labels):
            targets.append(i)
    if len(targets) == num_class:
        gmm.means_init = np.array([kmer_table[finalDf.Labels == i].mean(axis=0) for i in targets])
    gmm.fit(kmer_table)
    predictions = gmm.predict(kmer_table)

    # Get the counts for the given labels and the predicted labels
    given_labels_count = {}
    labels_list = list(labels)
    for label in unique_given_labels:
        given_labels_count[label] = labels_list.count(label)
    unique_predicted_labels = get_unique_numbers(predictions)
    predicted_labels_count = {}
    for label in unique_predicted_labels:
        predicted_labels_count[label] = (predictions == label).sum()
    max_item = max(predicted_labels_count, key=predicted_labels_count.get)
    if -1 in given_labels_count.keys():
        del given_labels_count[-1]
    given_labels_count = sorted(given_labels_count.items(), key=lambda x: x[1], reverse=True)
    predicted_labels_count = sorted(predicted_labels_count.items(), key=lambda x: x[1], reverse=True)

    res = np.array(predictions)

    # Map the predicted labels to the given/actual labels
    unselected_given = copy.deepcopy(unique_given_labels)
    if -1 in unselected_given:
        unselected_given.remove(-1)
    unselected_pred = copy.deepcopy(unique_predicted_labels)
    map_predict_to_actual = {}
    for label_GIVEN_dict_entry in given_labels_count:
        label_GIVEN = label_GIVEN_dict_entry[0]
        predicted_labels_count_GIVEN = {}
        label_GIVEN_idx = [index for (index, item) in enumerate(labels_list) if item == label_GIVEN ]
        res_GIVEN = [res[i] for i in label_GIVEN_idx]
        unique_predicted_labels_GIVEN = list(set(get_unique_numbers(res_GIVEN)) & set(unselected_pred))
        if len(unique_predicted_labels_GIVEN) == 0:
            continue
        for lab in unique_predicted_labels_GIVEN:
            predicted_labels_count_GIVEN[lab] = (res_GIVEN == lab).sum()
        map_predict_to_actual[max(predicted_labels_count_GIVEN, key=predicted_labels_count_GIVEN.get)] = label_GIVEN
        unselected_given.remove(label_GIVEN)
        to_rem = max(predicted_labels_count_GIVEN, key=predicted_labels_count_GIVEN.get)
        unselected_pred.remove(to_rem)
        print("remove ", label_GIVEN, "from unselected given. ", unselected_given, "remains")
        print("remove ", to_rem, "from unselected pred. ", unselected_pred, "remains")


    # in the case where multiple given labels completely map to the same 
    # predicted label, we need to finish assigning given labels to any 
    # predicted label
    for lab_remain in unselected_given:
        for upl in unique_predicted_labels:
            if upl not in map_predict_to_actual.keys():
                map_predict_to_actual[upl] = lab_remain
                unselected_given.remove(lab_remain)
                unselected_pred.remove(upl)
                break

    if len(unique_given_labels) < num_class:
        max_value = max(unique_given_labels) + 1
        for upl in unique_predicted_labels:
            if upl not in map_predict_to_actual.keys():
                # print(f"{upl} mapped to {max_value}")
                map_predict_to_actual[upl] = max_value
                max_value += 1
                unselected_pred.remove(upl)
    
    if len(unselected_given) != len(unselected_pred):
        print("unselected given != unselected pred")
        print("unselected pred: ", unselected_pred)
        print("unselected given: ", unselected_given)
    
    
    print(f"map_predict_to_actual: {map_predict_to_actual}")
    predictions_final = []
    predictions_tmp = []

    # predictions_final contains the final results
    # it takes care of the case when num_class > number of unique labels given
    for i in range(len(predictions)):
        if labels[i] == -1:
            if predictions[i] in map_predict_to_actual.keys():
                predictions_final.append(map_predict_to_actual[predictions[i]])
                predictions_tmp.append(map_predict_to_actual[predictions[i]])
            else:
                predictions_final.append(map_predict_to_actual[max_item])
                predictions_tmp.append(map_predict_to_actual[max_item])
        else:
            predictions_tmp.append(map_predict_to_actual[predictions[i]])
            predictions_final.append(labels[i])

    # get accuracy with regard to known labels

    unknown_label = -1
    total_labeled = 0
    for i in range(len(labels)):
        if labels[i] != unknown_label:
            total_labeled = total_labeled + 1

    correct_count = 0
    temp_accuracy = 0
    for k in range(len(labels)):
        if (labels[k] != unknown_label):
            if (labels[k] == predictions_tmp[k]):
                correct_count += 1
    temp_accuracy = correct_count / total_labeled


    predictions = np.array(predictions_final)
    output_df.insert(0, "Labels", predictions)
    return predictions,temp_accuracy


def get_unique_numbers(numbers):
    list_of_unique_numbers = []

    unique_numbers = set(numbers)

    for number in unique_numbers:
        list_of_unique_numbers.append(number)

    return list_of_unique_numbers

def model_selection(path,labels,num_class):
    best_accu = 0
    best_prediction = []
    cov_type = ['full','diag','tied','spherical']
    k_min = [2,3,4,5]
    k_max = [2,3,4,5]
    for cov in cov_type:
        for k1 in k_min:
            for k2 in k_max:
                if (k2 >= k1):
                    prediction,accu = get_predictions_semi(path,k1,k2,num_class,cov,0,labels)
                    #accu = cal_accuracy(labels,prediction)
                    if accu > best_accu: 
                        best_accu = accu
                        best_kmin = k1
                        best_kmax = k2
                        best_cov = cov
                        best_prediction = prediction
    print('Best model has the following parameters:')
    print('minimum length of kmer: ', best_kmin)
    print('maximum length of kmer: ', best_kmax)
    print('covariance type: ', best_cov)
    print('It has an accuracy regard to known labels of ',best_accu)
    return best_prediction

# Nucleotides

In [23]:
import os
print(os.getcwd())

/Users/samoz/Documents/masters/research/bioklustering2/bioklustering/new_analysis_2024/scripts


In [24]:
path = "../combined_nucleotide.fasta"
labels_nuc = pd.read_csv("../combined_labels_nucleotide.csv")
labels_nuc = pd.Series(labels_nuc['Labels'])
k_min = 2
k_max = 3 
num_class = 2 
cov_type = 'full' 
seed = 1232
predictions_nuc, tmp = get_predictions_semi(path, k_min, k_max, num_class, cov_type, seed, labels_nuc) #model_selection(path,labels_nuc,num_class)

df_nuc, output_df = get_kmer_table(path,k_min,k_max)
x = df_nuc
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents,columns = ['principal component 1', 'principal component 2'])
finalDf = pd.concat([pd.Series(output_df.index), principalDf, pd.Series(predictions_nuc)], axis = 1)
finalDf.columns = ['ID','principal Component 1', 'principal Component 2','label']
finalDf.to_csv('../results/nuc_gmm_pca.csv',index = False)

remove  0 from unselected given.  [1] remains
remove  0 from unselected pred.  [1] remains
remove  1 from unselected given.  [] remains
remove  1 from unselected pred.  [] remains
map_predict_to_actual: {0: 0, 1: 1}


In [25]:
n_unlab = sum(labels_nuc == -1)

print("Ones: ", sum(predictions_nuc), " out of ", n_unlab)
print("Zeros: ", sum(1- predictions_nuc), " out of ", n_unlab)

Ones:  201  out of  350
Zeros:  165  out of  350


# Amino Acids

In [26]:
path = "../combined_amino.fasta"
labels_am = pd.read_csv("../combined_labels_amino.csv")
labels_am = pd.Series(labels_am['Labels'])
k_min = 2
k_max = 3
num_class = 2 
cov_type = 'full' 
seed = 1232
predictions_am, tmp = get_predictions_semi(path, k_min, k_max, num_class, cov_type, seed, labels_am) #model_selection(path,labels_am,num_class)

df_am, output_df = get_kmer_table(path,k_min,k_max)
x = df_am
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents,columns = ['principal component 1', 'principal component 2'])
finalDf = pd.concat([pd.Series(output_df.index), principalDf, pd.Series(predictions_am)], axis = 1)
finalDf.columns = ['ID','principal Component 1', 'principal Component 2','label']
finalDf.to_csv('../results/am_gmm_pca.csv',index = False)

remove  0 from unselected given.  [1] remains
remove  1 from unselected pred.  [0] remains
remove  1 from unselected given.  [] remains
remove  0 from unselected pred.  [] remains
map_predict_to_actual: {1: 0, 0: 1}


In [27]:
n_unlab = sum(labels_am == -1)

print("Ones: ", sum(predictions_am), " out of ", n_unlab)
print("Zeros: ", sum(1- predictions_am), " out of ", n_unlab)

Ones:  444  out of  465
Zeros:  37  out of  465
